In [40]:
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from statistics import mean
from numpy.linalg import norm
from pathlib import Path

## Load Human Prompts

In [41]:
human_prompt_files = [
    "../sentence_embeddings/data/amazonTurkDesPrompt1.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt2.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt3.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt4.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt5.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt6.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt7.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt8.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt9.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt10.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt11.csv",
    "../sentence_embeddings/data/amazonTurkDesPromptNA.csv",
]

human_prompts = []

for human_prompt in human_prompt_files:
    prompts = []
    with open(human_prompt, 'r') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            prompts.append(row[0])
    prompts = [i.replace('\n',' ') for i in prompts]
    human_prompts.append(prompts)

In [42]:
pd.DataFrame(human_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,ï»¿A jump rope that is also to be used as a ti...,A muscle stimulate band that can be worn over ...,"A simple, but strong band that has a hook to b...","A calf strengthener, similar to a finger stren...",A rubber ball that can be inflated / deflated ...,An x shaped resistance band that has 2 hand gr...,A mat that has electrosensitive pressure point...,A fold bike that could be used as a bike and a...,I think there should be a product that will he...,I am thinking of a portable exercise device.,...,A vest that makes you flex your chest muscles ...,A fold up tension devise that can fit into a s...,something very small that will allow you to wo...,A rowing type machine that can be folded up in...,This device is a dumbbell in which the weighte...,a small collapsible step that can be used to d...,How about an abdominal or muscle training MP3 ...,"Flexible, non-electric, ergonomic, compact","A collapsible, compact elliptical machine.",I am thinking of some sort of rubber band stro...
1,ï»¿A vest made of crystal quartz pieces connec...,Perhaps there could be something...a cord?...a...,a device which is attached to the joints so wh...,"When you blush, you put sensors on your cheeks...",People could wear kinetic bracelets and anklet...,solar equipment collects energy,A bicycle-powered television,have electrodes that can suction cup to our bo...,A battery that is charged by the energy from y...,A shoe that collects the energy generated when...,...,Im thinking of clothing that contians statice ...,I know skin conductance goes up when people ar...,A stationary bike with the wheel connected to ...,put pressure buttons in the bottoms of shoes a...,I am thinking of something simple like a watch...,sort of like roller blades that store the mech...,Energy from the motion of walking could be col...,A smart phone app that trackA smart phone app ...,"To collect energy from human motion, a station...",Power collecting shoes that joggers or top rac...
2,ï»¿with a candle that has marks on it,A clock or new redesigned clock that measures ...,waterfall clock tracking time by water ammount,Instead of relying on visual inputs these woul...,A small chart on your wrist,A giant hourglass,A clock that used binary to measure time.,Marking a piece of string with knots to measur...,A medication that lets you be so in tune with ...,a clock that acts more like a measure of the s...,...,"""Timeline"" is a flexible strip of LED lights e...",Digital self-drawn laser clock art. The design...,Use base 10 instead of base-6 numbers.,A block you put in the freezer and it turns bl...,We could divide time up into different color q...,The sun app. The sun app uses a satellite in ...,It would be the number of times my fan turned ...,A device that monitors the rotations and posit...,TIME IS INFINITE THEREFORE SOMETHING HAS TO BE...,light bulbs that dim as the sun goes down
3,"ï»¿""A tall cylindrical bottle","Like a party pop, this would be for celebratio...",a turn table which senses something underneath...,It would be a puffer kind of like a perfume bo...,Use the same container that weed killer comes ...,I thought of an automatic powdered sugar dispe...,Conveyor belt through a box that sprays the po...,A metal powder shaker with small circular hole...,An electrically powered sifter. Like a hand po...,A small long object that looks like a scanner....,...,A series of spinning rotors move several spray...,"This would be like a car wash, but instead of ...",a cannister that sprays and coats something of...,A small canister with a spigot at the top and ...,A container with a perforated lid to dispense ...,A small salt-shaker like item. This one has a ...,"A cup/tube cover, with multiple layers that ha...",I bake often at home and this prompt made me i...,A device that can be used while cooking and ba...,An automatic brush machine that could be adjus...
4,"ï»¿""A retractable gripper that has a central h...",A rotating shelf

## Load GPT Prompts

In [43]:
def get_all_files(directory, pattern):
    return [f for f in Path(directory).glob(pattern)]

In [44]:
gpt_files = get_all_files("../prompt_engineering/data/few_shot/", "*.csv")
len(gpt_files)

60

In [45]:
gpt_prompts = []

# for design_prompt_count in range(11):
for iteration_count in range(5):
    iteration_prompts = []
    # for iteration_count in range(5):
    for design_prompt_count in range(12):
        fail = False
        prompt = pd.read_csv(f"../prompt_engineering/data/few_shot/design_prompt_{design_prompt_count}_{iteration_count}.csv", sep=",") #, header=None)
        lst = []
        for d in prompt.values:
            lst.append(d[0])
        prompt = []
        for i in range(len(lst)):
            try:
                mod_statements = lst[i].replace(lst[i][:3], '')
                prompt.append(mod_statements)
            except AttributeError:
                fail = True
        if fail:
            print(f"Could not process design_prompt_{design_prompt_count}_{iteration_count}")

        iteration_prompts.append(prompt)
    gpt_prompts.append(iteration_prompts)

Could not process design_prompt_1_0
Could not process design_prompt_0_1
Could not process design_prompt_0_2
Could not process design_prompt_10_2
Could not process design_prompt_9_3
Could not process design_prompt_10_3
Could not process design_prompt_10_4


In [46]:
pd.DataFrame(gpt_prompts)

,0,1,2,3,4,5,6,7,8,9,10,11
0,[A collapsible set of weights that have a buil...,[A backpack that contains a motorized pulley s...,[Mechanically powered wall clock with a 24-hou...,"[A handheld device with a reservoir of powder,...","[""A rotary-style robotic arm mounted on the ce...",[A powerful frother attachment that can be use...,[Phone cases that have a built in strap so it ...,[A hands-free device that takes the clothes/to...,[Have the fountain mounted to the wall at an a...,[The measuring cup would be equipped with a sp...,[A knee brace that wraps around the joint and ...,[A device that automatically dims electrical l...
1,[A core activator belt that can be worn to act...,[A generator that fits around the waist and us...,[An hourglass that measures the passage of tim...,"[Like a fog machine, this would be able to dis...","[A extendable ladder with a hook on the end., ...",[A portable handheld frother that runs from an...,[Having a device that works with smart watches...,[A foldable metal frame with two arms that can...,[Install a longer-length flexible drinking hos...,"[A tactile measuring cup with raised numbers.,...",[A metal structure with adjustable joints to i...,[An automated system to adjust the thermostat ...
2,[A handheld device with elastic bands and a ha...,[a small weighted wheel that rotates when you ...,"[mechanical gears clock that tracks time, robo...",[An automated robotic arm that moves around th...,[A motorized robotic arm that sits in the spac...,[A manual contraption that fits into a standar...,[Place signs in public areas warning pedestria...,[A laundry machine with inbuilt folding capabi...,[Make the fountain accessible to both wheelcha...,[The measuring cup has a setting to vibrate wh...,[A passive restraint device with straps and bu...,[A thermostat system that tracks and maintains...
3,[An adjustable resistance band that wraps arou...,[A device that attaches to your arms and legs ...,[A wristwatch that measures the passage of tim...,[It would be a spray bottle that sprays a fine...,[A telescoping pole with a basket on the end t...,[A milk frothing pitcher with a built-in motor...,[A text alert that reminds users to be aware o...,[Hand-cranked machine with flat surface and ar...,[Install a drinking fountain with disability-f...,[Provide a measuring cup with Braille identifi...,[A device that employs infrared sensors and ai...,[A device that uses the sun’s energy to heat o...
4,[A retractable jump rope that is stored in an ...,[A lightweight elastic fitness band that can b...,[A bracelet with tiny beads spaced out at each...,[A handheld device that has a nozzle that shoo...,[A telescopic “grabber” unit with an extendabl...,[a battery-powered whisk that creates a vortex...,[Install sensors in the road to detect cell ph...,[An automated wall-mounted device that folds t...,[A built in water filter to provide clean drin...,[The measuring cup has a series of tactile mar...,[A device that uses an exoskeleton of rods and...,[A device that detects when our home exceeds i...


In [47]:
pd.DataFrame(gpt_prompts[0])

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,A collapsible set of weights that have a built...,A Yoga mat that folds into a small bag with a ...,A set of collapsible resistance bands that can...,A pair of hand weights that are designed to fi...,A mini-elliptical that folds up and is designe...,A folding travel-weight lifting bench that has...,A collapsible pull-up bar that attaches to the...,A set of travel-sized dumbbells that are light...,A jump rope that can be wrapped around a suit...,Yoga blocks made out of foam that can easily ...,...,A jump rope that can be wrapped around a back...,A set of foam blocks that are adjustable for ...,A yoga mat with straps that can be used for a...,A set of balance discs that can be stored in ...,A chest press machine that can be stored in a...,A doorway pull-up bar with adjustable tension...,A folding exercise bench designed to fit in a...,A handheld device with adjustable weights to ...,A balance ball that can be adjusted to differ...,. A resistance tunnel that is easy to store an...
1,A backpack that contains a motorized pulley sy...,A pair of shoes with a built-in kinetic energy...,An adjustable arm band made of a flexible mate...,A pair of kinetic energy harvesting wrist band...,A stationary bike with a built-in solar panel....,An adjustable elastic belt with a built-in kin...,A hand crank generator attached to an exercise...,A set of hand weights with a built-in generato...,A weighted vest with a built-in kinetic energ...,An exercise machine with a built-in kinetic e...,...,None,None,None,None,None,None,None,None,None,None
2,Mechanically powered wall clock with a 24-hour...,Cuckoo clock that has five minutes increment,Sun-dial with shadows of the sun used to measu...,Color-coded clock that display the time in hue...,Clock that is powered by sound waves from a so...,Personal digital assistant (PDA) with timer,Digital watch with LCD display and multiple al...,Digital clock that is synchronized with the TV...,Wall-mounted timer which is programmable and ...,Smartphone apps that track elapsed time,...,Digital wristwatch with a light-up display,Wristwatch with a compass and timer,Pocket watch with a 24-hour display,Smartwatch with a pedometer and timer,Digital clock with a programmable timer,Digital timer with a rechargable battery,Digital timer with a touch screen,Automated clock with a voice recognition system,Digital timer with a solar-powered LCD display,. Hand-held timer with a digital display
3,"A handheld device with a reservoir of powder, ...",A bucket or container with a conveyor belt att...,"A hopper mounted on a framework, above the sur...","A backpack style device, with a reservoir of p...",A wall-mounted device with a reservoir of powd...,"A handheld device with a reservoir of powder, ...","A pump-action device, with a reservoir of powd...",A stand-mounted device with a reservoir of pow...,"A container with a funnel, filled with powder...",A container with a spigot and a perforated li...,...,A container with a lid and a number of small ...,A motorized device with a reservoir of powder...,A handheld device with a reservoir of powder ...,A stand-mounted device with a reservoir of po...,A tall cylindrical bottle with a spout on the...,A hand-held device with a reservoir of powder...,"A backpack style device, with a reservoir of ...","A handheld device with a reservoir of powder,...",A handheld device with a reservoir of powder ...,. A stand-mounted device with a reservoir of p...
4,"""A rotary-style robotic arm mounted on the cei...","""A telescopic arm with a grabber at the end. T...","""A mobile robotic book retrieval system. It wi...","""A wall-mounted robotic arm that will lift and...","""A telescopic rod with a robotic arm at the en...","""A mobile robotic book retrieval system with a...","""A wall-mounted robotic arm with a magnetized ...","""A robotic arm with a vacuum suction cup. The ...","""An articulated arm with a spring-loaded claw...","""An adjustable ro

## Embed data

In [48]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [49]:
human_embeddings = [model.encode(prompt) for prompt in human_prompts]

gpt_embeddings_iterations = []
for iteration in gpt_prompts:
    gpt_embeddings_iterations.append([model.encode(prompt) for prompt in iteration])

In [50]:
np.array(gpt_embeddings_iterations).shape

C:\Users\grandid\AppData\Local\Temp\1\ipykernel_55524\2590467071.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(gpt_embeddings_iterations).shape


(5, 12)


# Similarity Metrics
## Nearest Generated Sample
Using cosine distance
Focus on design space coverage

In [51]:
distances_all = []

for embedding in gpt_embeddings_iterations:
    distances_model = []
    for count, generated_embedding in enumerate(embedding):
        distances = []
        for original_datapoint in human_embeddings[count]:
            max_distance = -1e9
            for generated_datapoint in generated_embedding:
                distance = np.dot(original_datapoint, generated_datapoint)/(norm(original_datapoint)*norm(generated_datapoint))
                if distance > max_distance:
                    max_distance = distance
            distances.append(max_distance)
        mean_distance = mean(distances)
        # print(f"Mean distance of nearest generated samples for prompt {count} to human: {mean_distance}")
        distances_model.append(mean_distance)
    distances_all.append(distances_model)


In [52]:
distances_pd = pd.DataFrame(distances_all, index=range(5)).transpose()
distances_pd.style.background_gradient()

,0,1,2,3,4
0,0.627114,0.607393,0.562197,0.639350,0.606247
1,0.585320,0.639346,0.600368,0.627129,0.632547
2,0.527584,0.600213,0.536957,0.565999,0.539008
3,0.565822,0.548775,0.635790,0.658715,0.629024
4,0.633661,0.689637,0.682826,0.662271,0.673035
5,0.670159,0.667160,0.660797,0.660311,0.693853
6,0.619632,0.633647,0.614082,0.588461,0.617993
7,0.683686,0.675421,0.679342,0.636916,0.685366
8,0.650402,0.623654,0.634307,0.594744,0.618794
9,0.702319,0.722276,0.610386,0.658365,0.701942


In [53]:
distances_pd.mean(axis=1)

0     0.608460
1     0.616942
2     0.553952
3     0.607625
4     0.668286
5     0.670456
6     0.614763
7     0.672146
8     0.624380
9     0.679058
10    0.575578
11    0.617017
dtype: float32

### Mean distance

In [54]:
distances_all = []

for embedding in gpt_embeddings_iterations:
    distances_model = []
    for count, generated_embedding in enumerate(embedding):
        distances = []
        for original_datapoint in human_embeddings[count]:
            max_distance = -1e9
            for generated_datapoint in generated_embedding:
                distance = np.dot(original_datapoint, generated_datapoint) / (
                            norm(original_datapoint) * norm(generated_datapoint))
                distances.append(distance)
        mean_distance = mean(distances)
        # print(f"Mean distance of nearest generated samples for prompt {count} to human: {mean_distance}")
        distances_model.append(mean_distance)
    distances_all.append(distances_model)

distances_pd = pd.DataFrame(distances_all, range(5)).transpose()
distances_pd.style.background_gradient()

,0,1,2,3,4
0,0.372530,0.392267,0.389239,0.447118,0.365505
1,0.443753,0.511920,0.405486,0.482308,0.459516
2,0.293067,0.412406,0.303294,0.410021,0.366695
3,0.439801,0.430949,0.457495,0.513490,0.521246
4,0.548053,0.369348,0.527755,0.445517,0.562875
5,0.547830,0.492322,0.501844,0.551451,0.525897
6,0.407751,0.441370,0.359449,0.350939,0.407933
7,0.510794,0.561851,0.556203,0.441682,0.595650
8,0.487680,0.373928,0.454524,0.455119,0.369838
9,0.494741,0.553807,0.512802,0.543941,0.515843


### Mean distance with combinatorial selection

In [56]:
from itertools import combinations

distances_all = []

for embedding in gpt_embeddings_iterations:
    distances_prompt = []
    for count in range(len(embedding)):
        distances_row = []
        iterate = list(range(len(np.array(embedding[count]))))
        max_distance = -1e9
        for combo in combinations(iterate, 2):
            # dot product
            distance = np.dot(human_embeddings[count][combo[0],:], embedding[count][combo[1],:])/(norm(human_embeddings[count][combo[0],:])*norm(embedding[count][combo[1],:]))
            if distance > max_distance:
                max_distance = distance
            distances_row.append(max_distance)
        distances_prompt.append(np.mean(distances_row))
    distances_all.append(distances_prompt)

In [57]:

distances_pd = pd.DataFrame(distances_all, range(5)).transpose()
distances_pd.style.background_gradient()

,0,1,2,3,4
0,0.803382,0.831761,0.641473,0.777952,0.811662
1,0.670526,0.784065,0.778102,0.795430,0.685583
2,0.696181,0.780873,0.757348,0.802845,0.739251
3,0.713660,0.704148,0.783050,0.767813,0.763004
4,0.775471,0.818587,0.811613,0.785756,0.807854
5,0.807617,0.821751,0.817344,0.811116,0.812543
6,0.792017,0.803559,0.808472,0.706480,0.836358
7,0.826030,0.857638,0.842654,0.808053,0.877208
8,0.813626,0.835568,0.777360,0.618698,0.740768
9,0.872596,0.888945,0.786405,0.712135,0.839304


In [58]:
distances_pd.mean(axis=0)

0    0.793338
1    0.814227
2    0.788413
3    0.770352
4    0.783919
dtype: float32

# Novelty/Diversity Metrics
## Convex hull hypervolume

In [59]:
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
from tqdm import tqdm

In [60]:
np.array(human_embeddings).shape

(12, 100, 384)

In [61]:
np.array([human_embeddings] + gpt_embeddings_iterations).shape

C:\Users\grandid\AppData\Local\Temp\1\ipykernel_55524\549932370.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([human_embeddings] + gpt_embeddings_iterations).shape


(6, 12)

In [62]:
pca = PCA(n_components=3)

volumes_all = []
for prompts in tqdm([human_embeddings] + gpt_embeddings_iterations):
    volumes_prompts = []
    for prompt in tqdm(prompts, leave=False):
        embeddings = pca.fit_transform(prompt)
        convex_hull = ConvexHull(embeddings)
        volume_prompt = convex_hull.volume
        volumes_prompts.append(volume_prompt)
    volumes_all.append(volumes_prompts)

100%|██████████| 6/6 [00:00<00:00,  7.22it/s] 


In [63]:
convex_hull_volumes_df = pd.DataFrame(volumes_all, index=['human', 'iteration 1', 'iteration 2', 'iteration 3', 'iteration 4', 'iteration 5']).transpose()
convex_hull_volumes_df.style.background_gradient()

,human,iteration 1,iteration 2,iteration 3,iteration 4,iteration 5
0,0.249209,0.272238,0.185465,0.178628,0.144559,0.255028
1,0.243629,0.139101,0.136639,0.226110,0.115445,0.196254
2,0.321649,0.295191,0.132038,0.173457,0.191801,0.265659
3,0.317053,0.141965,0.190726,0.211800,0.139574,0.117120
4,0.338390,0.056472,0.349411,0.220953,0.220270,0.077826
5,0.261036,0.083188,0.123826,0.196066,0.105052,0.240811
6,0.330279,0.247635,0.245501,0.301907,0.265899,0.243826
7,0.276590,0.142680,0.064899,0.126344,0.294472,0.075083
8,0.327088,0.167231,0.373654,0.352700,0.132803,0.251590
9,0.306567,0.280266,0.091362,0.096986,0.112691,0.114206


In [64]:
convex_hull_volumes_df.mean(axis=0)

human          0.294032
iteration 1    0.186456
iteration 2    0.195887
iteration 3    0.207756
iteration 4    0.162059
iteration 5    0.169529
dtype: float64

In [65]:
convex_hull_volumes_df.mean(axis=1)

0     0.214188
1     0.176196
2     0.229966
3     0.186373
4     0.210554
5     0.168330
6     0.272508
7     0.163345
8     0.267511
9     0.167013
10    0.149055
11    0.226401
dtype: float64